In [15]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [16]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [17]:
url = requests.get(URL)

In [18]:
soup = BeautifulSoup(url.content, "lxml") 
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrbwNApAIDAAAG7YrNgAAADK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"

In [64]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [65]:
# drop rows with 'Not assigned' in Borough column
df = df[df.Borough != 'Not assigned'].reset_index()

In [66]:
df.head(10)

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,Islington Avenue
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [67]:
df.shape

(103, 4)

In [68]:
df['Neighborhood'] = df['Neighborhood'].str.replace("/", ",")

In [69]:
df.loc[:, 'Borough']

0            North York
1            North York
2      Downtown Toronto
3            North York
4      Downtown Toronto
             ...       
98            Etobicoke
99     Downtown Toronto
100        East Toronto
101           Etobicoke
102           Etobicoke
Name: Borough, Length: 103, dtype: object

In [70]:
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
%pip install geocoder
import geocoder

Solving environment: done

# All requested packages already installed.

You should consider upgrading via the '/Users/kitaeklee/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [77]:
# Use Postal Code to get latitude and logitude and apply to the new columns


df['latitude']=""
df['longitude']=""

y=0
for x, z in zip(df.loc[:,'Borough'], df.loc[:,'Neighborhood']):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode("{}, {}, Toronto, Ontario, Canada".format(x,y ), timeout=None)
    df.loc[y, 'latitude'] = location.latitude
    df.loc[y, 'longitude'] = location.longitude
    y+=1
    if y==103:
        break



/Users/kitaeklee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead of None to get rid of this warning.
  # Remove the CWD from sys.path while we load stuff.


In [78]:
df


,index,Postal Code,Borough,Neighborhood,latitude,longitude
0,2,M3A,North York,Parkwoods,43.7543,-79.4491
1,3,M4A,North York,Victoria Village,43.7543,-79.4491
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6533,-79.3646
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543,-79.4491
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6402,-79.4382
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6715,-79.5525
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.6563,-79.3809
100,168,M7Y,East Toronto,Business reply mail Processing Centre,43.6248,-79.3935
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6715,-79.5525


,index,Postal Code,Borough,Neighborhood,latitude,longitude
0,2,M3A,North York,Parkwoods,43.7543,-79.4491
1,3,M4A,North York,Victoria Village,43.7543,-79.4491
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6533,-79.3646
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543,-79.4491
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6402,-79.4382
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6715,-79.5525
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.6563,-79.3809
100,168,M7Y,East Toronto,Business reply mail Processing Centre,43.6248,-79.3935
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6715,-79.5525


In [95]:
# create map of Toronto using latitude and longitude values
location_toronto = geolocator.geocode("Toronto, Ontario, Canada")
map_toronto = folium.Map(location=[location_toronto.latitude, location_toronto.longitude], zoom_start=10)

df.loc[:, 'latitude'].astype('float')
df.loc[:, 'longitude'].astype('float')

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [91]:
# Define Foursquare Credentials and Version
CLIENT_ID = '3MAR2Y4AH5PUQKHSI4TCMUQMLU2S45MO0TVJAKMHDDHAGINK' #  Foursquare ID
CLIENT_SECRET = 'SRPNGW0J3R5EFCSNVENLIXE4BUCCI0X2EEDBYRZTGYLY4HL3' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT = 50

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [92]:
# def function on each neighborhood and create a new dataframe
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [100]:
toronto_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.754326,-79.449117,Grill Gate,43.753123,-79.451690,Mediterranean Restaurant
1,Parkwoods,43.754326,-79.449117,Orly Restaurant & Grill,43.754493,-79.443507,Middle Eastern Restaurant
2,Parkwoods,43.754326,-79.449117,Tim Hortons,43.754767,-79.443250,Coffee Shop
3,Parkwoods,43.754326,-79.449117,Crave Restaurant,43.753133,-79.450378,Wings Joint
4,Victoria Village,43.754326,-79.449117,Grill Gate,43.753123,-79.451690,Mediterranean Restaurant


In [118]:
# Analyze each neighborhood
# one hot encoding —> Converting categorical vairalbes to numeric variables
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot = toronto_onehot.set_index('Neighborhood').reset_index()


In [119]:
toronto_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
# Group rows by neighborhood and by taking the mean of the frequency of occurence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.166667,0.0,0.00,0.0,0.0
1,"Alderwood, Long Branch",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0
3,Bayview Village,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Newtonbrook",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0
94,Woburn,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,...,0.0,0.0,0.023256,0.0,0.023256,0.000000,0.0,0.00,0.0,0.0
95,Woodbine Heights,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0
96,York Mills West,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0


In [121]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                           venue  freq
0                    Coffee Shop  0.17
1  Paper / Office Supplies Store  0.17
2                     Taco Place  0.17
3          Vietnamese Restaurant  0.17
4                           Bank  0.17


----Alderwood, Long Branch----
         venue  freq
0   Playground   0.2
1  Supermarket   0.2
2     Pharmacy   0.2
3  Coffee Shop   0.2
4       Garden   0.2


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Wings Joint  0.25
1  Middle Eastern Restaurant  0.25
2                Coffee Shop  0.25
3   Mediterranean Restaurant  0.25
4        American Restaurant  0.00


----Bayview Village----
                       venue  freq
0                Wings Joint  0.25
1  Middle Eastern Restaurant  0.25
2                Coffee Shop  0.25
3   Mediterranean Restaurant  0.25
4        American Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                       venue  freq
0     

                       venue  freq
0                Wings Joint  0.25
1  Middle Eastern Restaurant  0.25
2                Coffee Shop  0.25
3   Mediterranean Restaurant  0.25
4        American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0       Clothing Store  0.08
1  American Restaurant  0.04
2   Seafood Restaurant  0.04
3       Cosmetics Shop  0.04
4                 Café  0.04


----Leaside----
              venue  freq
0               Pub  0.17
1  Greek Restaurant  0.17
2      Liquor Store  0.17
3              Park  0.17
4       Pastry Shop  0.17


----Little Portugal, Trinity----
                 venue  freq
0       Clothing Store  0.08
1  American Restaurant  0.04
2   Seafood Restaurant  0.04
3       Cosmetics Shop  0.04
4                 Café  0.04


----Malvern, Rouge----
            venue  freq
0  Clothing Store  0.16
1  Cosmetics Shop  0.07
2        Pharmacy  0.05
3     Coffee Shop  0.05
4      Shoe Store  0.05


----Milliken, Agincourt North, Steeles

                       venue  freq
0                Wings Joint  0.25
1  Middle Eastern Restaurant  0.25
2                Coffee Shop  0.25
3   Mediterranean Restaurant  0.25
4        American Restaurant  0.00


----York Mills, Silver Hills----
                       venue  freq
0                Wings Joint  0.25
1  Middle Eastern Restaurant  0.25
2                Coffee Shop  0.25
3   Mediterranean Restaurant  0.25
4        American Restaurant  0.00




In [122]:
# Write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [123]:
# Create a new dataframe and display top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


In [125]:
#  Cluster neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 1, 3, 3, 3, 0, 0, 0, 4, 0], dtype=int32)

In [130]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged

,index,Postal Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M3A,North York,Parkwoods,43.7543,-79.4491,3,Wings Joint,Middle Eastern Restaurant,Coffee Shop,Mediterranean Restaurant,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space
1,3,M4A,North York,Victoria Village,43.7543,-79.4491,3,Wings Joint,Middle Eastern Restaurant,Coffee Shop,Mediterranean Restaurant,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6533,-79.3646,0,Coffee Shop,Italian Restaurant,Restaurant,Theater,Breakfast Spot,Yoga Studio,Karaoke Bar,Cosmetics Shop,Dessert Shop,Discount Store
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543,-79.4491,3,Wings Joint,Middle Eastern Restaurant,Coffee Shop,Mediterranean Restaurant,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6402,-79.4382,0,Bakery,Tibetan Restaurant,Café,Pizza Place,Diner,Restaurant,Indian Restaurant,Pharmacy,Sandwich Place,Caribbean Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6715,-79.5525,1,Garden,Coffee Shop,Supermarket,Playground,Pharmacy,Cuban Restaurant,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.6563,-79.3809,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Plaza,Theater,Cosmetics Shop,Modern European Restaurant
100,168,M7Y,East Toronto,Business reply mail Processing Centre,43.6248,-79.3935,4,Historic Site,Sculpture Garden,Music Venue,Boat or Ferry,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Fish Market
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6715,-79.5525,1,Garden,Coffee Shop,Supermarket,Playground,Pharmacy,Cuban Restaurant,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop


In [133]:
# Visualize
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters